In [ ]:
# Ocultar log de instalacao
%%capture cap_output

In [ ]:
!git clone https://github.com/hygo2025/cdd-projeto-final.git

In [ ]:
# Setup spark
!apt install openjdk-21-jdk

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.699/aws-java-sdk-bundle-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.12.699/aws-java-sdk-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core/1.12.699/aws-java-sdk-core-1.12.699.jar" -P "spark/lib-jars/"
!wget "https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-s3/1.12.699/aws-java-sdk-s3-1.12.699.jar" -P "spark/lib-jars/"

In [47]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

os.environ["SPARK_DRIVER_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_EXECUTOR_EXTRACLASSPATH"] = "/content/spark/lib-jars/*"
os.environ["SPARK_SERIALIZER"] = "org.apache.spark.serializer.KryoSerializer"
os.environ["ENV"] = "dev"
os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["SPARK_EXECUTOR_MEMORY"] = "8g"
os.environ["SPARK_DRIVER_MEMORY"] = "8g"
os.environ["SPARK_MEMORY_FRACTION"] = "8g"


os.environ["DOWNLOAD_FOLDER"] = "/content/datasets"

In [18]:
!java --version

openjdk 21.0.6 2025-01-21
OpenJDK Runtime Environment (build 21.0.6+7-Ubuntu-122.04.1)
OpenJDK 64-Bit Server VM (build 21.0.6+7-Ubuntu-122.04.1, mixed mode, sharing)


In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
try:  # When on Google Colab, clone the repository to download any necessary cache.
    import google.colab
    repo_path = 'cdd'
    !git -C $repo_path pull origin || git clone https://github.com/hygo2025/cdd-projeto-final.git $repo_path
except:
    repo_path = '.'  # Use the local path if not on Google Colab

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8 (delta 5), reused 8 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 854 bytes | 284.00 KiB/s, done.
From https://github.com/hygo2025/cdd-projeto-final
   bc74ed6..840d6fd  main       -> origin/main
Updating bc74ed6..840d6fd
Fast-forward
 settings.toml                       | 2 ++
 src/dataset/movielens/downloader.py | 6 ++++--
 src/dataset/movielens/loader.py     | 5 ++++-
 3 files changed, 10 insertions(+), 3 deletions(-)


In [22]:
import sys
import os

if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
!pip install -r cdd/requirements.txt

In [ ]:
import pkg_resources
!pip install -e $repo_path

Ignoring spacy: markers 'python_version <= "3.8"' don't match your environment
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is ta

In [49]:
import cdd

In [50]:
from src.sar_model import SarModel
# from src.utils.enums import MovieLensDataset, SimilarityType

# sar_model = SarModel(
#     dataset=MovieLensDataset.ML_100K,
#     top_k=10,
#     validate_size=0.25,
#     time_decay_coefficient=30,
#     similarity_type=SimilarityType.COSINE,
#     seed=42
# )
# sar_model.train()
# result = sar_model.evaluate()
# print("Resultados da avaliação:")
# print(result)

ModuleNotFoundError: No module named 'recommenders'